In [1]:
import math
import torch
import matplotlib.pyplot
from torch import autograd, nn

In [2]:
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # Get the subnetwork by sorting the scores and using the top k%
        out = scores.clone()
        _, idx = scores.flatten().sort()
        j = int((1-k) * scores.numel())
        
        # flat_out and out access the same memory.
        flat_out = out.flatten()
        flat_out[idx[:j]] = 0
        flat_out[idx[j:]] = 1
        return out
    
    @staticmethod
    def backward(ctx, g):
    # send the gradient g straight-through on the backward pass.
        return g, None

class SubnetConv(nn.Conv2d):
    # self.k is the % of weights remaining, a real number in [0,1]
    # self.popup_scores is a Parameter which has the same shape as self.weight
    # Gradients to self.weight, self.bias have been turned off.
    def forward(self, x):
        # Get the subnetwork by sorting the scores.
        adj = GetSubnet.apply(
        self.popup_scores.abs(), self.k)
        # Use only the subnetwork in the forward pass.
        w = self.weight * adj
        x = F.conv2d(
            x, w, self.bias, self.stride, self.padding,
                self.dilation, self.groups
        )
        return x